this part of the code is where i:

- extract features from the data using sklearn
- train ml models to predict the total amount of shoes expected to be sold
- possibly engineer new data that the model will be trained on

In [ ]:
from sklearn.preprocessing import train_test_split, LabelEncoder
from sklearn.metrics import confusion_matrix, f1_score, balanced_accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier

from lazypredict.Supervised import LazyRegressor